### to implement later: 
* clean the affiliations so that the country is moved into its own column

* standardize the paths so that everything is more logical

* create full dataset based on scientists

* change so that multiple countries can flow into one publication

* some data viz: show how many pubmedIDs per organism; how many origins over the years

* Entrez download: For now I open the sequences of each species separately, but later on this should all be in one dataset (once I implemented the Entrez download)

* Add author, institution, country to the cleaned data of sequences (either as a list in a column, or by create various entries for each author attribution of a sequence - then I should add an index independent of the ANs)


In [1]:
from Bio import SeqIO
import collections
import pandas as pd 
import re
import requests
import json
import time
import collections
import glob
import pprint
import seaborn as sns
pp = pprint.PrettyPrinter()

# 1. Genbank data

## 1.1. Opening the downloaded sequence data and exporting the cleaned data + origin-publicaiton data

In [20]:
path = "C:/Users/pauld/Documents/GitHub/Genetic sequence flows/Data/Raw data/Genetic sequences"
all_files = glob.glob(path + "/*.gb")

#standardize the organisms: 
replacements = { 'Takifugu rubripes rubripes' : 'Takifugu rubripes', 'Pyrococcus furiosus COM1' : 'Pyrococcus furiosus', 'Thermotoga maritima MSB8' : 'Thermotoga maritima', 'Pyrococcus furiosus DSM 3638':'Pyrococcus furiosus'}
replacer = replacements.get

#create data for each organism in loop: 
for filename in all_files:
    for gb_record in SeqIO.parse(open(filename,"r"), "genbank"):  
        gb_record?
        print(gb_record.features)
        
        #print(gb_record.annotations['organism'])
        
        break
    

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(4483), strand=1), type='source'), SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(4483), strand=1), type='gene'), SeqFeature(FeatureLocation(ExactPosition(136), ExactPosition(3487), strand=1), type='CDS')]
[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(402), strand=1), type='source')]
[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(1724609), strand=1), type='source'), SeqFeature(FeatureLocation(BeforePosition(0), ExactPosition(674), strand=1), type='gene'), SeqFeature(FeatureLocation(BeforePosition(0), ExactPosition(674), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(834), ExactPosition(2193), strand=1), type='gene'), SeqFeature(FeatureLocation(ExactPosition(834), ExactPosition(2193), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(2607), ExactPosition(3354), strand=-1), type='gene'), SeqFeature(FeatureLocation(ExactPosition(2607), ExactPosition(3354), str

In [49]:
# read the gb data and make csvs
# this code is for all sequences without break (because there appear to be very little overlaps) ## 'what are overlaps`?
# # can I parse only those that already have references? 
# may want to see whether I can detect more author institutions than only through the Lens


# determine the datafiles
path = "C:/Users/pauld/Documents/GitHub/Genetic sequence flows/Data/Raw data/Genetic sequences"
all_files = glob.glob(path + "/*.gb")

#standardize the organisms: 
replacements = { 'Takifugu rubripes rubripes' : 'Takifugu rubripes', 'Pyrococcus furiosus COM1' : 'Pyrococcus furiosus', 'Thermotoga maritima MSB8' : 'Thermotoga maritima', 'Pyrococcus furiosus DSM 3638':'Pyrococcus furiosus'}
replacer = replacements.get

#create data for each organism in loop: 
for filename in all_files:
    print(filename)
    
    pubmed_ids = []
    sequence_ANs = []
    origins = []
    organisms = []
    all_pubmed_ids = [] # may want to create one big file that also includes missing values, maybe that would be faster than making to separate lists for pubmedIDs (one with country matches and one without)
    all_sequence_ANs = []
    all_origins = []
    all_sequence_pubdates = []
    all_organisms = []
    #institutions = [] this could be possible through the 'journal field in references[1], but maybe this would be easier through Lens'


    m = 0
    count = 1

    for gb_record in SeqIO.parse(open(filename,"r"), "genbank"):  
        
        # create full data lists: 
        country = 'none' # this is not really needed, only in case I want to show countries and non-countries
        for feat in gb_record.features:
                if feat.type == 'source':
                        source = gb_record.features[0]
                        for qualifiers in source.qualifiers:
                            if 'country' in qualifiers:
                                country = re.findall("^(?:(?!:)(?!def).)*",str(source.qualifiers['country'][0]))[0]
        all_origins.append(country)
        
        
        all_sequence_ANs.append(gb_record.name)
        all_sequence_pubdates.append(str(gb_record.annotations['date'])[-4:])
        all_organisms.append(gb_record.annotations['organism'])
        
        
        medID = 'none' 
        if 'references' in list(gb_record.annotations.keys()): 
            if gb_record.annotations['references'][0].pubmed_id:
                medID = gb_record.annotations['references'][0].pubmed_id
        all_pubmed_ids.append(medID) 
        
        # now go into the matches for the network data (match it based on whether country of origin info is available)
        if 'references' in list(gb_record.annotations.keys()): 
            if gb_record.annotations['references'][0].pubmed_id:
                for feat in gb_record.features:
                        if feat.type == 'source': #take only those with countries --> unecessary if I want to have both with and without country
                            source = gb_record.features[0]
                            for qualifiers in source.qualifiers:
                                if 'country' in qualifiers:
                                    country = re.findall("^(?:(?!:)(?!def).)*",str(source.qualifiers['country'][0]))[0]
                                    pubmed_ids.append(gb_record.annotations['references'][0].pubmed_id)
                                    sequence_ANs.append(gb_record.name)
                                    origins.append(country)  
                                    organisms.append(gb_record.annotations['organism'])
        
                                        
                                    m = m+1

                                    #if m == 60:  # I only need this if there is sufficient overlapping sequences

    # export network data
    organisms[:] = [replacer(n, n) for n in organisms]
    dictdata_network = {'origin': origins, 'pubMed': pubmed_ids, 'ANs': sequence_ANs, 'Organisms': organisms} # maybe I have to write it as dict instead of dictdata? 
    df = pd.DataFrame(dictdata_network)
    df.to_csv('C:/Users/pauld/Documents/GitHub/Pilot study/Network data/Separate organisms/'+str(gb_record.annotations['organism'])+'_country-pubmedID_edges_'+str(m)+'.csv',index=False)
    
    # export full data
    dictdata_full = {'AN':all_sequence_ANs, 'pubMed': all_pubmed_ids, 'origins':all_origins, 'pub. date': all_sequence_pubdates, 'organism': all_organisms}
    df_full = pd.DataFrame(dictdata_full)
    df_full.to_csv('C:/Users/pauld/Documents/GitHub/Pilot study/Cleaned full data/'+str(gb_record.annotations['organism'])+'_fulldata.csv',index=False)

C:/Users/pauld/Documents/GitHub/Genetic sequence flows/Data/Raw data/Genetic sequences\sequences[takifugu+rubripes].gb
C:/Users/pauld/Documents/GitHub/Genetic sequence flows/Data/Raw data/Genetic sequences\sequences[tetraodon nigroviridis].gb
C:/Users/pauld/Documents/GitHub/Genetic sequence flows/Data/Raw data/Genetic sequences\sequence[photobacterium sp. ska34].gb
C:/Users/pauld/Documents/GitHub/Genetic sequence flows/Data/Raw data/Genetic sequences\sequence[pyrococcus furiosus].gb
C:/Users/pauld/Documents/GitHub/Genetic sequence flows/Data/Raw data/Genetic sequences\sequence[thermotoga maritima].gb


In [3]:
# this is to merge the different datasets and make one out of it. could merge that with the code above
path = 'C:/Users/pauld/Documents/GitHub/Pilot study/Network data/Separate organisms'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    print(filename)
    df_ed = pd.read_csv(filename, encoding= 'unicode_escape', dtype = 'string') #, index_col=None, header=0
    li.append(df_ed)

df_edges_orig_pub_combined = pd.concat(li, axis=0, ignore_index=True)


df_edges_orig_pub_combined.to_csv('C:/Users/pauld/Documents/GitHub/Pilot study/Network data/orig_pub_edges.csv',index=False)

ValueError: No objects to concatenate